# Datathon — Treinamento do Modelo CatBoost

Pipeline de treinamento do modelo de alerta preventivo de risco educacional. Consome os datasets de features gerados pelo `feature_engineering.ipynb` e produz um modelo treinado pronto para avaliação e inferência.

---

## Visão Geral

| Etapa | Descrição |
|---|---|
| **1. Carregamento** | Lê `train_feat` e `valid_feat` gerados no `feature_engineering.ipynb` |
| **2. Imports** | Verifica instalação do CatBoost e importa dependências |
| **3. Preparação** | Tipagem correta de categóricas e numéricas; montagem dos Pools CatBoost |
| **4. Hiperparâmetros** | Configuração do `CatBoostClassifier` com early stopping e balanceamento de classes |
| **5. Treinamento** | Roda com `eval_set=valid_pool` e early stopping out-of-time |
| **6. Predição e Avaliação** | Probabilidade, score 0–100 e AUC na validação temporal |
| **7. Exportação** | Salva modelo em `.joblib` (bundle completo) e `.cbm` (formato nativo CatBoost) |

---

## Decisões de Design

| Decisão | Justificativa |
|---|---|
| **CatBoostClassifier** | Suporte nativo a variáveis categóricas sem one-hot encoding manual; robusto em datasets tabulares de médio porte |
| **`auto_class_weights="Balanced"`** | Corrige o desbalanceamento entre positivos (piora) e negativos (estável/melhora) sem resampling |
| **Early stopping na validação temporal** | Para o treino quando a validação out-of-time parar de melhorar — evita overfitting ao par 2022→2023 |
| **`use_best_model=True`** | Retorna o checkpoint com melhor AUC na validação, não o estado final das iterações |
| **Subsampling de linhas (`0.8`) e colunas (`rsm=0.8`)** | Regularização implícita — reduz correlação entre as árvores e melhora generalização |

---

## Saídas deste Notebook

| Arquivo | Conteúdo |
|---|---|
| `catboost__piora__2022_2023.joblib` | Bundle completo: modelo + `feature_cols` + `cat_features` + `params` |
| `catboost__piora__2022_2023.cbm` | Modelo no formato binário nativo do CatBoost (para servir via API) |


## 1. Carregamento dos Datasets de Features

Lê os Parquets produzidos pelo `feature_engineering.ipynb` — já com todas as variáveis construídas, sem leakage e com schema idêntico entre treino e validação.

| Arquivo | Par | Conteúdo |
|---|---|---|
| `train_feat__piora__2022_2023.parquet` | 2022→2023 | Features + `target` de treino |
| `valid_feat__piora__2023_2024.parquet` | 2023→2024 | Features + `target` de validação out-of-time |


In [29]:
from pathlib import Path
import pandas as pd

# Caminhos locais
ROOT_DIR = Path("..").resolve()          # notebooks/ -> workspace root
NOTEBOOKS_DIR = ROOT_DIR / "notebooks"  # onde os parquets de features estão

train_feat = pd.read_parquet(NOTEBOOKS_DIR / "data" / "train_feat__piora__2022_2023.parquet")
valid_feat = pd.read_parquet(NOTEBOOKS_DIR / "data" / "valid_feat__piora__2023_2024.parquet")

print("✅ Features carregadas de:", NOTEBOOKS_DIR / "data")
print("  train_feat:", train_feat.shape, "  valid_feat:", valid_feat.shape)


✅ Features carregadas de: /home/glauberthy/Desktop/datathon/notebooks/data
  train_feat: (600, 122)   valid_feat: (765, 122)


## 2. Imports e Dependências

In [30]:
# catboost já instalado no ambiente local (.venv)
# Para instalar manualmente: pip install catboost
import catboost
print("✅ catboost version:", catboost.__version__)


✅ catboost version: 1.2.10


In [31]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score

### Dependências

| Biblioteca | Papel |
|---|---|
| `CatBoostClassifier` | Modelo de gradient boosting com suporte nativo a categóricas |
| `Pool` | Estrutura de dados do CatBoost — carrega features, rótulos e metadados de tipo em uma única chamada |
| `roc_auc_score` | Métrica de avaliação global — probabilidade de ranquear corretamente positivo vs. negativo |


## 3. Preparação dos Dados para o Modelo

In [32]:
TARGET = "target"
ID_COLS = ["ra","ano_base","pair_label"] # colunas que não devem entrar no modelo (identificadores e metadados do split).
CAT_COLS = ["fase","turma","genero","instituicao"] # olunas categóricas que o CatBoost deve tratar como categoria.

### 3.1 Definição de Features, IDs e Target

Separa explicitamente o papel de cada coluna para evitar que identificadores ou metadados contaminem o modelo:

| Grupo | Colunas | Uso |
|---|---|---|
| **Target** | `target` | Variável resposta (1 = piora, 0 = estável/melhora) |
| **IDs** | `ra`, `ano_base`, `pair_label` | Rastreabilidade — excluídos das features |
| **Categóricas** | `fase`, `turma`, `genero`, `instituicao` | Passadas ao CatBoost como índices de coluna categórica |
| **Numéricas** | demais colunas | Tratadas como `float64` |


In [33]:
feature_cols = [c for c in train_feat.columns if c not in ID_COLS + [TARGET]]
cat_features = [c for c in CAT_COLS if c in feature_cols]
num_features = [c for c in feature_cols if c not in cat_features]

X_train = train_feat[feature_cols].copy()
y_train = train_feat[TARGET].astype(int)

X_valid = valid_feat[feature_cols].copy()
y_valid = valid_feat[TARGET].astype(int)

### 3.2 Tratamento de Tipos — Categóricas

O CatBoost exige que colunas categóricas sejam do tipo `object` ou `string` — sem `NaN`. Valores ausentes são substituídos por `"MISSING_CATEGORY"`, que o modelo aprende a tratar como categoria própria, preservando a informação de ausência.


In [34]:
for col in cat_features:
    X_train[col] = X_train[col].astype('object').fillna('MISSING_CATEGORY').astype('category')
    X_valid[col] = X_valid[col].astype('object').fillna('MISSING_CATEGORY').astype('category')

/tmp/ipykernel_194936/3924895106.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train[col] = X_train[col].astype('object').fillna('MISSING_CATEGORY').astype('category')
/tmp/ipykernel_194936/3924895106.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_valid[col] = X_valid[col].astype('object').fillna('MISSING_CATEGORY').astype('category')


### 3.3 Tratamento de Tipos — Numéricas

Garante que todas as colunas numéricas estão como `float64`. Valores não parseáveis (strings residuais) viram `NaN` via `errors="coerce"` — o CatBoost lida nativamente com `NaN` em numéricas.


In [35]:
for col in num_features:
    X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
    X_valid[col] = pd.to_numeric(X_valid[col], errors='coerce')

### 3.4 Construção dos Pools CatBoost

`Pool` é a estrutura de entrada preferida do CatBoost. Passar as colunas categóricas como **índices numéricos** (em vez de nomes) evita ambiguidade quando colunas são reordenadas e é obrigatório para uso com `eval_set` via `Pool`.


In [36]:
cat_feature_indices = [X_train.columns.get_loc(col) for col in cat_features]
train_pool = Pool(X_train, y_train, cat_features=cat_feature_indices)
valid_pool = Pool(X_valid, y_valid, cat_features=cat_feature_indices)

## 4. Configuração do Modelo — Hiperparâmetros

| Parâmetro | Valor | Justificativa |
|---|---|---|
| `iterations` | 2500 | Limite superior de árvores — early stopping decide o ponto ideal |
| `learning_rate` | 0.03 | Taxa baixa combinada com muitas iterações → melhor generalização |
| `depth` | 8 | Árvores profundas o suficiente para capturar interações, mas ainda regularizadas |
| `auto_class_weights` | `"Balanced"` | Compensa desbalanceamento sem necessidade de resampling |
| `bootstrap_type` + `subsample` | Bernoulli / 0.8 | Amostra 80 % das linhas por árvore — reduz variância |
| `rsm` | 0.8 | Amostra 80 % das features por split — semelhante ao Random Forest, aumenta diversidade |
| `l2_leaf_reg` | 6.0 | Regularização L2 nas folhas — penaliza pesos extremos |
| `early_stopping_rounds` | 200 | Para quando AUC na validação não melhora por 200 rodadas consecutivas |
| `eval_metric` | `"AUC"` | Métrica de parada é AUC — coerente com a métrica de avaliação final |


In [37]:
params = dict(
    iterations=2500,
    learning_rate=0.03,
    depth=8,
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=42,
    auto_class_weights="Balanced",
    bootstrap_type="Bernoulli",
    subsample=0.8,
    rsm=0.8,
    l2_leaf_reg=6.0,
    early_stopping_rounds=200,
    verbose=200
)

## 5. Treinamento

Treina usando `train_pool` e monitora `valid_pool` a cada iteração. O early stopping interrompe automaticamente quando a AUC de validação para de melhorar. `use_best_model=True` garante que o modelo retornado corresponde ao checkpoint de maior AUC na validação — não ao estado da última iteração.


In [38]:
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=valid_pool, use_best_model=True)

0:	test: 0.4085887	best: 0.4085887 (0)	total: 2.77ms	remaining: 6.92s
200:	test: 0.7068125	best: 0.7095408 (197)	total: 624ms	remaining: 7.14s
400:	test: 0.6949730	best: 0.7151460 (344)	total: 1.27s	remaining: 6.64s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7151460318
bestIteration = 344

Shrink model to first 345 iterations.


CatBoostClassifier(auto_class_weights='Balanced', bootstrap_type='Bernoulli', depth=8, early_stopping_rounds=200, eval_metric='AUC', iterations=2500, l2_leaf_reg=6.0, learning_rate=0.03, loss_function='Logloss', random_seed=42, rsm=0.8, subsample=0.8, verbose=200)

## 6. Predição e Avaliação

### 6.1 Probabilidade e Score

`predict_proba` retorna a probabilidade de `target = 1` (piora) para cada aluno. O `valid_score` é a mesma probabilidade convertida para escala 0–100, usada pelo pipeline operacional para ranquear alunos e aplicar a política Top-K%.


In [39]:
valid_proba = model.predict_proba(valid_pool)[:,1]
valid_score = (100*valid_proba).round(4)

### 6.2 Avaliação Global — AUC

In [40]:
auc = roc_auc_score(y_valid, valid_proba)
print("✅ AUC (valid):", round(auc, 6))

✅ AUC (valid): 0.715146


#### Interpretação do AUC

A **AUC (Area Under the ROC Curve)** mede a capacidade discriminativa do modelo: dado um aluno que vai piorar e um que não vai, a AUC é a probabilidade de o modelo ranquear corretamente o mais arriscado.

| AUC | Interpretação prática |
|---|---|
| 0.5 | Equivalente a chute aleatório — sem poder preditivo |
| 0.6–0.7 | Discriminação fraca — útil apenas como baseline |
| **0.7–0.8** | **Discriminação moderada — aceitável para triagem educacional** |
| 0.8–0.9 | Discriminação boa |
| > 0.9 | Excelente (suspeitar de leakage se muito próximo de 1.0) |

> A AUC é a métrica de referência neste projeto, mas a **avaliação operacional** real (Recall@TopK, Lift@K, Precision@K) é feita no `evaluate.ipynb`, que simula a política de intervenção Top-K% estratificada por fase.


## 7. Exportação do Modelo

O modelo é salvo em dois formatos complementares:

| Formato | Arquivo | Uso |
|---|---|---|
| `.joblib` | `catboost__piora__2022_2023.joblib` | Bundle completo — carrega modelo + `feature_cols` + `cat_features` + `params` em uma única chamada; usado pelo `evaluate.ipynb` e pela API |
| `.cbm` | `catboost__piora__2022_2023.cbm` | Formato binário nativo do CatBoost — portável, independente da versão do Python; usado para servir via `model.load_model()` |


In [41]:
import joblib
from pathlib import Path

BASE_DIR = NOTEBOOKS_DIR / "models"  # modelos salvos em notebooks/models/
BASE_DIR.mkdir(parents=True, exist_ok=True)

JOBLIB_PATH = BASE_DIR / "catboost__piora__2022_2023.joblib"

bundle = {
    "model": model,
    "feature_cols": feature_cols,
    "cat_features": cat_features,
    "target": "target",
    "params": params,
}

joblib.dump(bundle, JOBLIB_PATH)

print("✅ Modelo salvo em joblib:", JOBLIB_PATH)


✅ Modelo salvo em joblib: /home/glauberthy/Desktop/datathon/notebooks/models/catboost__piora__2022_2023.joblib


In [42]:
CBM_PATH = BASE_DIR / "catboost__piora__2022_2023.cbm"
model.save_model(str(CBM_PATH))